# Step 1: Define set of halos we want to investigate.

### Steps:
* Query for Halos: shows usage of CasJobs.
* Store them in HDF5 and CSV file : shows usage of Teporary file storage
* Store them in a table : shows usage of MyDB
* Share CSV file :  shows usage of Files Service 
<!-- <img src="images/uc1.png"/> -->

In [1]:
# Import necessary libraries.
import SciServer.CasJobs as cj
import os
import pandas
import tables

In [2]:
# define SQL: query from Millennium halo merger trees tab 
# find halos with mass in given range
query = """
SELECT haloId,np,x,y,z, halfMassRadius 
  FROM mpahalotrees..mr 
 WHERE snapnum=63 
   AND np between 200000 AND 210000
"""

# query CasJobs table. Using MPAHaloTrees as context
halosDF=cj.executeQuery(query, "MPAHaloTrees")

print("Found",len(halosDF),"halos")

Found 102 halos


In [3]:
# this code is here to adjust the root_folder scratch storage to the user
import SciServer.Authentication as auth
USER=auth.getKeystoneUserWithToken(auth.getToken())
root_folder=f'/home/idies/workspace/Temporary/{USER.userName}/scratch/cosmodata/'

In [4]:
# store result as HDF5 file 
os.makedirs(root_folder,exist_ok=True)
h5store = pandas.HDFStore(f'{root_folder}HaloSample.h5')
h5store['halos']=halosDF
h5store.close()

# store result as CSV file
halosDF.to_csv(f'{root_folder}HaloSample.csv')

GOTO root folder and see data has appeared

GOTO CasJobs schema browser and ensure no table halo_sample

create table in MyDB

In [5]:
# in case the table was already created:
try:
    cj.executeQuery("drop table halo_sample","MyDB")    
except:
    pass

ddl = """
CREATE TABLE halo_sample(
  haloID bigint,np integer,x real,y real,z real, halfMassRadius real)
"""
cj.executeQuery(ddl,"MyDB")

,Rows Affected
0,0


upload directly from DataFrame

In [6]:
response=cj.uploadPandasDataFrameToTable(halosDF,"halo_sample")

GOTO CasJobs and see that table was written